In [29]:
from os import listdir
import os
import numpy as np
import operator

In [1]:
def MNNB(sub_dict, words, total):
    prob = -1
    for word in words:
        if(word in sub_dict):
            prob += (np.log2(sub_dict[word] + 1) - np.log2(sub_dict["total_count"] + total))
    prob += (np.log2(sub_dict["total_count"]) - np.log2(total))
    return prob

In [2]:
def find_prediction(dictionary, file_path):
    l_words = read_file(file_path)
    words = clean_data(l_words)
    prob = -1
    best_prob = -1000000
    news_grp = ''
    dict_keys = dictionary.keys()
    for key in dict_keys:
        if(key != 'total'):
            prob = MNNB(dictionary[key], words, dictionary["total"])
            print(prob)
            if(prob > best_prob):
                news_grp = key
                best_prob = prob
    return news_grp

In [3]:
def predict(dictionary, path):
    files_path = os.listdir(path)
    predictions = []
    for file_path in files_path:
        pred = find_prediction(dictionary, path + "\\" + file_path)
        predictions.append(pred)
    return predictions

In [4]:
def files_path(base_path):
    folders_name = listdir(base_path)
    folders_path = []
    for folder_name in folders_name:
        folders_path.append(base_path + "\\" + folder_name)
    file_path = []
    for folder_path in folders_path:
        l = os.listdir(folder_path)
        for i in l:
            file_path.append(folder_path + "\\" + i)
    return folders_name, folders_path, file_path

In [5]:
def clean_data(l):
    stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", " ", ".", ",", ":", ";", "-", "--", "?", "/", "\\", '']
    for word in list(l):
        if(word in stopwords or not word.isalpha()):
            l.remove(word)
    return l

In [6]:
def read_file(path):
    l = []
    i = 0
    myfile = open(path, 'r')
    for line in myfile.readlines():
        if (i > 12):
            x = (line.strip().split(" "))
            for w in x:
                l.append(w.lower())
        i += 1
    return l

In [7]:
def build_word_dictionary(list_words):
    dictionary_words = {}
    for word in list_words:
        if(word not in dictionary_words.keys()):
            dictionary_words[word] = 1
        else:
            dictionary_words[word] += 1
    return dictionary_words

In [8]:
def make_final_dict(dict_imp_words, folders_name, folders_path):
    dictionary = {}
    wd_keys = dict_imp_words.keys()
    for i in range(len(folders_name)):
        feature = folders_name[i]
        dictionary["total"] = len(dict_imp_words)
        dictionary[feature] = dict_imp_words.copy()
        path = folders_path[i]
        folder_i_files = os.listdir(path)
        for folder_i_file in folder_i_files:
            file_words = read_file(path + "\\" + folder_i_file)
            pure_words = clean_data(file_words)
            for word in pure_words:
                if(word not in wd_keys):
                    continue
                else:
                    dictionary[feature][word] += 1
        dictionary[feature]["total_count"] = sum(dictionary[feature].values())
    return dictionary

In [9]:
def dict_build(base_path, n):
    folders_name, folders_path, file_path = files_path(base_path)
    list_words = []
    for file in file_path:
        file_words = read_file(file)
        pure_words = clean_data(file_words)
        list_words += pure_words
    dictionary_words = build_word_dictionary(list_words)
    sorted_dict = sorted(dictionary_words.items(), key=operator.itemgetter(1), reverse = True)
    imp_words = sorted_dict[0:n]
    dict_imp_words = {}
    for i in range(len(imp_words)):
        if(imp_words[i][0] not in dict_imp_words.keys()):
            x = imp_words[i][0]
            dict_imp_words[x] = 0
    dictionary = make_final_dict(dict_imp_words, folders_name, folders_path)
    return dictionary

In [39]:
dictionary = dict_build("C:\\Users\\abhishek\\Desktop\\Abhi\\Git_hub\\Text_Classification\\20_newsgroups_train", 5500)

In [40]:
dictionary

{'total': 5500,
 'alt.atheism': {'i': 3256,
  'just': 474,
  'like': 414,
  'people': 701,
  'know': 391,
  'think': 539,
  'article': 383,
  'use': 151,
  'does': 508,
  'x': 27,
  'new': 145,
  'good': 276,
  'make': 279,
  'time': 240,
  'did': 182,
  'want': 141,
  'say': 450,
  'way': 254,
  'used': 131,
  'need': 120,
  'really': 178,
  'believe': 354,
  'right': 127,
  'said': 263,
  'god': 551,
  'going': 99,
  'using': 61,
  'work': 78,
  'government': 30,
  'got': 72,
  'better': 170,
  'number': 75,
  'sure': 101,
  'read': 203,
  'come': 126,
  'things': 198,
  'problem': 92,
  'information': 59,
  'look': 118,
  'university': 63,
  'little': 123,
  'point': 180,
  'probably': 90,
  'data': 11,
  'different': 158,
  'file': 18,
  'years': 65,
  'available': 13,
  'set': 87,
  'long': 105,
  'tell': 96,
  'try': 128,
  'power': 48,
  'help': 45,
  'thing': 118,
  'best': 71,
  'program': 15,
  'lot': 54,
  'fact': 161,
  'question': 127,
  'space': 32,
  'windows': 0,
  'run

In [41]:
p = "C:\\Users\\abhishek\\Desktop\\Abhi\\Git_hub\\Text_Classification\\20_newsgroups_test\\alt.atheism"
pr = predict(dictionary, p)

-241.10017759775488
-292.1172174237145
-288.63463060757573
-291.01379729901174
-280.7633331531925
-287.18807845263734
-290.714773590303
-279.7240356979934
-279.77120072056357
-292.20882567726454
-291.99074918166957
-284.820117372548
-280.33503665403936
-284.66924734383724
-285.7439959485308
-262.632571090555
-266.0173277285902
-266.1505679708397
-265.60093432350783
-239.87171787330473
-614.6122033661024
-747.870690711765
-729.0045620868726
-740.4576348630285
-734.6622079412572
-760.1482311132442
-745.0324623204008
-713.7622058580348
-723.3108089081088
-711.7540926868425
-750.1815234380653
-752.5753044907907
-745.9375690055963
-716.3597549588733
-701.7034539879883
-714.7269535664674
-714.1441304124672
-722.9520069711795
-708.2010797833806
-610.4059068110473
-533.4658320431549
-611.6103310265561
-606.336401728825
-599.7843793479061
-602.4202681029036
-596.2440317750505
-629.77676507543
-574.2207180503408
-580.5186865257452
-593.631226222241
-598.5268537610596
-592.2331488006905
-596.0053

-582.7489634232112
-574.7121014973949
-577.8545536604026
-592.1541751879761
-582.4929710755678
-582.8303942475046
-584.0782091816652
-589.9960700716772
-574.6380161908997
-594.2914945176874
-614.0769154411032
-587.6193369133935
-574.2733528230331
-587.8054947996101
-587.4187686510894
-582.0885777575478
-593.4209167671318
-592.0318770808184
-603.1891711873492
-587.5830683362535
-1012.100142897182
-1113.1862638714008
-1107.58039442234
-1127.494958088233
-1113.9963911617244
-1141.347613305566
-1143.4217779358873
-1071.3883613462153
-1071.67945684056
-1090.122464928989
-1127.582505101463
-1076.1398060974232
-1104.1134527544282
-1079.7137361084115
-1098.431968613397
-1033.4159799097815
-1062.1970594153001
-1087.6673913781706
-1032.6276857131056
-994.1789138674694
-839.1056521827438
-923.8595167019045
-918.8530722839879
-934.7301644113649
-913.4415331731814
-940.801058920293
-932.1951485365773
-909.5450841390331
-902.6433901543645
-906.8442320760333
-918.3022344262754
-901.9578031829154
-903

-2227.950684035432
-2535.1452781141197
-2544.81328198087
-2550.175391089465
-2548.594483416998
-2582.3782388098048
-2647.5362360230756
-2508.1519180803507
-2533.206022541376
-2497.7273693318934
-2612.93866793753
-2472.880113593546
-2528.1208867327614
-2449.603602124991
-2506.014984007698
-2358.633531610288
-2489.6015874842456
-2528.9747983706343
-2458.108250234706
-2244.3148081904083
-563.3115617733384
-579.5630959691142
-598.531682801607
-586.4646356567735
-576.5263444863795
-611.541557772101
-577.9436829738551
-563.362527007752
-559.8889001333245
-584.592838014693
-594.3467394867033
-583.0383921388301
-572.1285306580905
-557.7338309594137
-563.0639494316147
-560.8572975859821
-562.3332157892987
-557.9002780784431
-566.2260930748582
-566.523134680766
-239.5358819027843
-270.22149678249093
-261.47738032462985
-265.2580251580362
-262.51735110797785
-268.40259355365146
-280.20952389718843
-264.3899081555251
-263.0135376746478
-266.56592911742587
-270.7317094772318
-269.4779041462011
-268

-936.8083894280035
-1007.8882325592609
-1030.1592886954115
-1041.3839954147784
-1032.8513082093293
-1044.0899519407433
-1042.433666367852
-1031.1976747386657
-1027.0178649247362
-1036.763611054592
-1062.0663922907524
-1013.5841730834492
-1020.7888018376807
-1006.6894114647401
-1016.9841978100419
-915.6872229065485
-1015.791764557037
-997.10592759135
-1009.9628561709557
-936.7588872102015
-2220.003810862234
-2405.0316669629406
-2416.7224759002934
-2400.485693560005
-2413.3922631780606
-2420.791467886185
-2450.2895734664967
-2403.792119106175
-2384.893432787437
-2388.7057982706374
-2479.1483094774558
-2348.480215448369
-2389.4624740619597
-2324.871633400685
-2329.029509045918
-2193.989562467852
-2350.7330393919656
-2338.0686189980834
-2344.2610612292547
-2164.118738498903
-435.0187597769683
-466.81424900440277
-465.1856472714663
-469.4643138611177
-471.96527165812205
-466.55037393836255
-483.9271108756102
-457.9214350496877
-446.95351499502885
-459.39314353661115
-470.83196411279465
-466

-46.317426893738556
-55.2164883573762
-51.12326162627414
-51.339015749463584
-50.54890379587482
-56.91173446529024
-55.16735366477777
-51.13973814055415
-51.41469418391383
-53.727407857136235
-53.82592546873177
-54.95646469374322
-52.5384222632272
-54.21297413137041
-55.22622697999314
-46.47658136033111
-54.6474663284305
-53.014862545659845
-54.40277008565782
-51.48897903206468
-523.9798875776348
-587.980518233662
-587.9026737123746
-595.4079261241753
-597.307156434627
-603.081558111354
-608.6965439398753
-573.7344385038422
-562.1123153931626
-573.7488908894721
-588.8446480495911
-580.6858726265913
-579.2370893572078
-560.5239333762372
-586.7579460702128
-528.3118584096777
-564.2155463937144
-567.3723503475427
-556.4967684431878
-536.4792645380294
-515.5850127325629
-638.2557091930535
-626.8524837671474
-627.0880557815802
-629.4628034861609
-649.0193841066459
-628.7603176388563
-618.7835407761694
-618.4274713513271
-614.0231887354944
-622.2730540155304
-622.2187514742047
-630.690457834

-42.44808598944887
-55.85284376662154
-57.290501941681086
-56.16230539360487
-59.53736745786742
-59.687999750328785
-58.30300270400835
-52.76013230701335
-53.339202041007646
-59.01772826352505
-57.232021057059114
-57.14849874579625
-55.44727437516901
-55.35748171381211
-53.781729579486296
-41.44176180037227
-51.39880842200072
-40.03587448393721
-46.28520833182445
-42.66089632088885
-1063.2092875371209
-1223.6075630726457
-1248.216113515079
-1239.6343292692093
-1244.0222676686299
-1236.430321262664
-1275.2932994446353
-1232.073321431207
-1235.8853422168277
-1269.8476583528234
-1291.436403055897
-1212.7277795610528
-1238.9409610218581
-1169.8270574556955
-1203.0145811369268
-1094.2100221464407
-1188.5636577273492
-1190.348034099381
-1174.3482358465544
-1083.450100374109
-583.6423197728368
-609.4617265735777
-618.8128175932928
-619.5743564507093
-622.1947756570964
-626.5608092406943
-638.1544445594691
-615.1800938495448
-616.4261251900859
-631.1229656632764
-638.5842190973561
-608.0706421

In [47]:
# find how many files have been predicted right out of 200
total = 0
# all the different groups of news
news_g = ["alt.atheism","comp.graphics","comp.os.ms-windows.misc","comp.sys.ibm.pc.hardware","comp.sys.mac.hardware","comp.windows.x","misc.forsale","rec.autos","rec.motorcycles","rec.sport.baseball","rec.sport.hockey","sci.crypt","sci.electronics","sci.med","sci.space","soc.religion.christian","talk.politics.guns","talk.politics.mideast","talk.politics.misc","talk.religion.misc"]
# number of files predicted correctly out of 200 from each group
for i in range(len(news_g)):
    for p in pr:
        if(p == news_g[i]):
            total += 1
    print(t)

115
117
118
119
119
119
119
122
124
125
125
125
126
131
133
150
154
160
161
200


In [43]:
len(news_g)

20